***Importa biblioteca de analise de dados***

In [1]:
import pandas as pd

## FUNÇÕES AUXILIARES PARA OTIMIZAÇÃO

Funções reutilizáveis para cálculos de SLA e análises de desempenho.

In [2]:
def calcular_sla_por_prioridade(df_corretivas, prioridade, nome_prioridade):
    """
    Calcula indicadores de SLA para uma prioridade específica.
    
    Retorna: dict com total, atendidas, fechadas e percentuais
    """
    # Filtrar por prioridade (aceita string ou lista)
    if isinstance(prioridade, list):
        df_prioridade = df_corretivas[df_corretivas['PRIORIDADE'].isin(prioridade)]
    else:
        df_prioridade = df_corretivas[df_corretivas['PRIORIDADE'] == prioridade]
    
    total = df_prioridade.shape[0]
    
    if total == 0:
        return {
            'total': 0,
            'atendidas': 0,
            'fechadas': 0,
            'perc_atendimento': 0,
            'perc_fechamento': 0,
            'dataframe': df_prioridade
        }
    
    atendidas = df_prioridade[df_prioridade['Estado tempo atendimento'] == 'Regular'].shape[0]
    fechadas = df_prioridade[df_prioridade['Estado tempo fechamento'] == 'Regular'].shape[0]
    
    return {
        'total': total,
        'atendidas': atendidas,
        'fechadas': fechadas,
        'perc_atendimento': round((atendidas / total) * 100, 2),
        'perc_fechamento': round((fechadas / total) * 100, 2),
        'dataframe': df_prioridade
    }


def calcular_indicadores_setor(df, quadro, tipo_manutenção):
    """
    Calcula indicadores de manutenção (corretivas/preventivas) para um setor.
    
    Retorna: dict com abertas, fechadas e percentual
    """
    df_setor = df[df['QUADRO DE TRABALHO'] == quadro]
    df_tipo = df_setor[df_setor['TIPO SERVIÇO'] == tipo_manutenção]
    
    total_abertas = df_tipo.shape[0]
    
    if tipo_manutenção == 'Manutenção Corretiva':
        # Corretivas: considerar apenas 'Fechada'
        total_fechadas = df_tipo[df_tipo['ESTADO'] == 'Fechada'].shape[0]
    else:
        # Preventivas: excluir 'Aberta' e 'Cancelada'
        total_fechadas = df_tipo[
            (df_tipo['ESTADO'] != 'Aberta') & 
            (df_tipo['ESTADO'] != 'Cancelada')
        ].shape[0]
    
    percentual = round((total_fechadas / total_abertas) * 100, 2) if total_abertas > 0 else 0
    
    return {
        'abertas': total_abertas,
        'fechadas': total_fechadas,
        'percentual': percentual,
        'df_abertas': df_tipo,
        'df_fechadas': df_tipo[(df_tipo['ESTADO'] != 'Aberta') & (df_tipo['ESTADO'] != 'Cancelada')] if tipo_manutenção != 'Manutenção Corretiva' else df_tipo[df_tipo['ESTADO'] == 'Fechada']
    }


def calcular_distribuicao_prioridades(total_corretivas, sla_emergente, sla_urgente, sla_pouco_urgente):
    """
    Calcula percentuais de distribuição de prioridades.
    """
    if total_corretivas == 0:
        return {'emergente': 0, 'urgente': 0, 'pouco_urgente': 0}
    
    return {
        'emergente': round((sla_emergente / total_corretivas) * 100, 2),
        'urgente': round((sla_urgente / total_corretivas) * 100, 2),
        'pouco_urgente': round((sla_pouco_urgente / total_corretivas) * 100, 2)
    }

print("✓ Funções auxiliares carregadas com sucesso")

✓ Funções auxiliares carregadas com sucesso


***Informa o local do arquivo de analise de dados do arkmeds e o transforma em um Dataframe***

In [3]:
# Defina aqui o nome do arquivo do mês a ser processado
caminho_arquivo = 'C:/Users/rafae/Desktop/Indicadores/Dezembro/ordens_servico_dezembro.xls'

# Carregar os dados
df = pd.read_excel(caminho_arquivo)
print(f"Arquivo carregado: {caminho_arquivo}")
print(f"Total de registros: {len(df)}")

Arquivo carregado: C:/Users/rafae/Desktop/Indicadores/Dezembro/ordens_servico_dezembro.xls
Total de registros: 702


### 📚 Documentação das Funções

**`calcular_sla_por_prioridade(df_corretivas, prioridade, nome_prioridade)`**
- **Propósito:** Calcula SLA (triagem e fechamento) para uma prioridade específica
- **Entrada:** DataFrame de corretivas, prioridade (str ou list), nome descritivo
- **Retorno:** Dict com total, atendidas, fechadas, percentuais e DataFrame filtrado
- **Uso:** Elimina repetição de código para cada prioridade (Emergente, Urgente, Pouco Urgente)

**`calcular_indicadores_setor(df, quadro, tipo_manutenção)`**
- **Propósito:** Calcula indicadores de abertura/fechamento para um setor e tipo de OS
- **Entrada:** DataFrame principal, nome do quadro, tipo de serviço
- **Retorno:** Dict com abertas, fechadas, percentual e DataFrames
- **Uso:** Consolida cálculos de corretivas e preventivas em uma única função

**`calcular_distribuicao_prioridades(total_corretivas, sla_emergente, sla_urgente, sla_pouco_urgente)`**
- **Propósito:** Calcula percentuais de distribuição de prioridades
- **Entrada:** Totais de cada categoria de prioridade
- **Retorno:** Dict com percentuais de cada prioridade
- **Uso:** Padroniza cálculo de distribuição para gráficos

---

In [4]:
# Teste rápido das funções
print("=" * 60)
print("🧪 TESTE DAS FUNÇÕES OTIMIZADAS")
print("=" * 60)

# Teste 1: calcular_indicadores_setor
teste_eng = calcular_indicadores_setor(df, 'Engenharia Clínica', 'Manutenção Corretiva')
print(f"\n✓ Função calcular_indicadores_setor:")
print(f"  Input: Engenharia Clínica, Manutenção Corretiva")
print(f"  Output: {teste_eng['abertas']} abertas, {teste_eng['fechadas']} fechadas")

# Teste 2: calcular_sla_por_prioridade  
teste_sla = calcular_sla_por_prioridade(
    df[df['QUADRO DE TRABALHO'] == 'Engenharia Clínica'][df['TIPO SERVIÇO'] == 'Manutenção Corretiva'],
    'Urgente',
    'Teste Urgente'
)
print(f"\n✓ Função calcular_sla_por_prioridade:")
print(f"  Input: Prioridade 'Urgente'")
print(f"  Output: {teste_sla['total']} OS, {teste_sla['perc_atendimento']}% atendimento")

# Teste 3: calcular_distribuicao_prioridades
teste_dist = calcular_distribuicao_prioridades(100, 10, 50, 40)
print(f"\n✓ Função calcular_distribuicao_prioridades:")
print(f"  Input: 100 total (10 emerg, 50 urg, 40 pouco)")
print(f"  Output: {teste_dist['emergente']}% emerg, {teste_dist['urgente']}% urg, {teste_dist['pouco_urgente']}% pouco")

print("\n" + "=" * 60)
print("✅ Todas as funções testadas com sucesso!")
print("=" * 60)

🧪 TESTE DAS FUNÇÕES OTIMIZADAS

✓ Função calcular_indicadores_setor:
  Input: Engenharia Clínica, Manutenção Corretiva
  Output: 20 abertas, 19 fechadas

✓ Função calcular_sla_por_prioridade:
  Input: Prioridade 'Urgente'
  Output: 11 OS, 63.64% atendimento

✓ Função calcular_distribuicao_prioridades:
  Input: 100 total (10 emerg, 50 urg, 40 pouco)
  Output: 10.0% emerg, 50.0% urg, 40.0% pouco

✅ Todas as funções testadas com sucesso!


C:\Users\rafae\AppData\Local\Temp\ipykernel_23204\76396658.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['QUADRO DE TRABALHO'] == 'Engenharia Clínica'][df['TIPO SERVIÇO'] == 'Manutenção Corretiva'],


***Informa quantas ordens de serviço corretivas foram abertas no total (engenharia clinica e predial)***

In [5]:
corretivas = df[df['TIPO SERVIÇO'] == 'Manutenção Corretiva']
total_corretivas_abertas = corretivas.shape[0]
print(total_corretivas_abertas)

93


***Informa quantas ordens de serviço corretivas foram fechadas (Engenharia clinica e predial)***

In [6]:
corretivas_fechadas = corretivas[corretivas['ESTADO'] == 'Fechada']
total_corretivas_fechadas = corretivas_fechadas.shape[0]
print(total_corretivas_fechadas)

89


***Informa quantas ordens de serviço preventivas foram abertas no total (engenharia clinica e predial)***

In [7]:
preventivas = df[df['TIPO SERVIÇO'] == 'Manutenção Preventiva']
total_preventivas_abertas = preventivas.shape[0]
print(total_preventivas_abertas)

484


***Informa quantas ordens de serviço preventivas foram fechadas (Engenharia Clinica e Predial)***

In [8]:
preventivas_fechadas =  preventivas[preventivas['ESTADO'] == 'Fechada']
total_preventivas_fechadas = preventivas_fechadas.shape[0]
print(total_preventivas_fechadas)

241


***Filtra pelo quadro de engenharia clinica***

In [9]:
engenharia_clinica = df[df['QUADRO DE TRABALHO'] == 'Engenharia Clínica']

## ENGENHARIA CLÍNICA - ANÁLISE CONSOLIDADA (OTIMIZADO)

In [10]:
# ========== ENGENHARIA CLÍNICA - CORRETIVAS ==========
eng_corretivas = calcular_indicadores_setor(df, 'Engenharia Clínica', 'Manutenção Corretiva')

# Atribuir variáveis para compatibilidade com código existente
corretivas_engenharia = eng_corretivas['df_abertas']
total_corretivas_criadas_engenharia = eng_corretivas['abertas']
corretivas_engenharia_fechadas = eng_corretivas['df_fechadas']
total_corretivas_engenharia_fechadas = eng_corretivas['fechadas']
percentual_corretivas_fechadas_engenharia = eng_corretivas['percentual']

# Corretivas atendidas no prazo
corretivas_engenharia_regular = corretivas_engenharia[corretivas_engenharia['Estado tempo atendimento'] == 'Regular']
total_corretivas_engenharia_fechadas_regular = corretivas_engenharia_regular.shape[0]
percentual_corretivas_fechadas_engenharia_regular = round(
    (total_corretivas_engenharia_fechadas_regular / total_corretivas_criadas_engenharia) * 100, 2
) if total_corretivas_criadas_engenharia > 0 else 0

print(f"✓ Corretivas Engenharia: {total_corretivas_criadas_engenharia} abertas, "
      f"{total_corretivas_engenharia_fechadas} fechadas ({percentual_corretivas_fechadas_engenharia}%)")
print(f"  → {total_corretivas_engenharia_fechadas_regular} atendidas no prazo ({percentual_corretivas_fechadas_engenharia_regular}%)")


# ========== ENGENHARIA CLÍNICA - PREVENTIVAS ==========
eng_preventivas = calcular_indicadores_setor(df, 'Engenharia Clínica', 'Manutenção Preventiva')

preventivas_engenharia = eng_preventivas['df_abertas']
total_preventivas_geradas_engenharia = eng_preventivas['abertas']
preventivas_engenharia_fechadas = eng_preventivas['df_fechadas']
total_preventivas_engenharia_fechadas = eng_preventivas['fechadas']
percentual_preventivas_fechadas_engenharia = eng_preventivas['percentual']

print(f"✓ Preventivas Engenharia: {total_preventivas_geradas_engenharia} abertas, "
      f"{total_preventivas_engenharia_fechadas} fechadas ({percentual_preventivas_fechadas_engenharia}%)")


# ========== ENGENHARIA CLÍNICA - SLA POR PRIORIDADE ==========
# SLA Emergente
sla_eng_emergente = calcular_sla_por_prioridade(corretivas_engenharia, 'Emergente', 'Emergente')
sla_triagem_emergente_engenharia = sla_eng_emergente['dataframe']
sla_triagem_emergente_total_engenharia = sla_eng_emergente['total']
sla_triagem_emergente_regular_total_engenharia = sla_eng_emergente['atendidas']
percentual_sla_triagem_emergente_engenharia = sla_eng_emergente['perc_atendimento']
sla_fechamento_emergente_regular_total_engenharia = sla_eng_emergente['fechadas']
percentual_sla_fechamento_emergente_engenharia = sla_eng_emergente['perc_fechamento']

# Variáveis para compatibilidade
sla_triagem_emergente_regular_engenharia = sla_triagem_emergente_engenharia[
    sla_triagem_emergente_engenharia['Estado tempo atendimento'] == 'Regular'
]
sla_fechamento_emergente_regular_engenharia = sla_triagem_emergente_engenharia[
    sla_triagem_emergente_engenharia['Estado tempo fechamento'] == 'Regular'
]

print(f"✓ SLA Emergente: {sla_triagem_emergente_total_engenharia} OS → "
      f"{percentual_sla_triagem_emergente_engenharia}% atendimento | "
      f"{percentual_sla_fechamento_emergente_engenharia}% fechamento")

# SLA Urgente
sla_eng_urgente = calcular_sla_por_prioridade(corretivas_engenharia, 'Urgente', 'Urgente')
sla_triagem_urgente_engenharia = sla_eng_urgente['dataframe']
sla_triagem_urgente_total_engenharia = sla_eng_urgente['total']
sla_triagem_urgente_regular_total_engenharia = sla_eng_urgente['atendidas']
percentual_sla_triagem_urgente_engenharia = sla_eng_urgente['perc_atendimento']
sla_fechamento_urgente_regular_total_engenharia = sla_eng_urgente['fechadas']
percentual_sla_fechamento_urgente_regular_total_engenharia = sla_eng_urgente['perc_fechamento']

# Variáveis para compatibilidade
sla_triagem_urgente_regular_engenharia = sla_triagem_urgente_engenharia[
    sla_triagem_urgente_engenharia['Estado tempo atendimento'] == 'Regular'
]
sla_fechamento_urgente_regular_engenharia = sla_triagem_urgente_engenharia[
    sla_triagem_urgente_engenharia['Estado tempo fechamento'] == 'Regular'
]

print(f"✓ SLA Urgente: {sla_triagem_urgente_total_engenharia} OS → "
      f"{percentual_sla_triagem_urgente_engenharia}% atendimento | "
      f"{percentual_sla_fechamento_urgente_regular_total_engenharia}% fechamento")

# SLA Pouco Urgente (inclui "Pouco urgente" e "Não urgente")
sla_eng_pouco = calcular_sla_por_prioridade(corretivas_engenharia, ['Pouco urgente', 'Não urgente'], 'Pouco Urgente')
sla_triagem_poucourgente_engenharia = sla_eng_pouco['dataframe']
sla_triagem_poucourgente_engenharia_total = sla_eng_pouco['total']
sla_triagem_poucourgente_regular_engenharia_total = sla_eng_pouco['atendidas']
percentual_sla_poucourgente_engenharia = sla_eng_pouco['perc_atendimento']
sla_fechamento_poucourgente_regular_total_engenharia = sla_eng_pouco['fechadas']
percentual_sla_fechamento_poucourgente_regular_total_engenharia = sla_eng_pouco['perc_fechamento']

# Variáveis para compatibilidade
sla_triagem_poucourgente_regular_engenharia = sla_triagem_poucourgente_engenharia[
    sla_triagem_poucourgente_engenharia['Estado tempo atendimento'] == 'Regular'
]
sla_fechamento_poucourgente_regular_engenharia = sla_triagem_poucourgente_engenharia[
    sla_triagem_poucourgente_engenharia['Estado tempo fechamento'] == 'Regular'
]

print(f"✓ SLA Pouco Urgente: {sla_triagem_poucourgente_engenharia_total} OS → "
      f"{percentual_sla_poucourgente_engenharia}% atendimento | "
      f"{percentual_sla_fechamento_poucourgente_regular_total_engenharia}% fechamento")


# ========== ENGENHARIA CLÍNICA - DISTRIBUIÇÃO DE PRIORIDADES ==========
dist_eng = calcular_distribuicao_prioridades(
    total_corretivas_criadas_engenharia,
    sla_triagem_emergente_total_engenharia,
    sla_triagem_urgente_total_engenharia,
    sla_triagem_poucourgente_engenharia_total
)

Percentagem_emergente = dist_eng['emergente']
percentagem_urgente = dist_eng['urgente']
percentagem_poucourgente = dist_eng['pouco_urgente']

print(f"\n✓ Distribuição de Prioridades:")
print(f"  → Emergente: {Percentagem_emergente}%")
print(f"  → Urgente: {percentagem_urgente}%")
print(f"  → Pouco Urgente: {percentagem_poucourgente}%")

✓ Corretivas Engenharia: 20 abertas, 19 fechadas (95.0%)
  → 12 atendidas no prazo (60.0%)
✓ Preventivas Engenharia: 80 abertas, 64 fechadas (80.0%)
✓ SLA Emergente: 0 OS → 0% atendimento | 0% fechamento
✓ SLA Urgente: 11 OS → 63.64% atendimento | 63.64% fechamento
✓ SLA Pouco Urgente: 9 OS → 55.56% atendimento | 55.56% fechamento

✓ Distribuição de Prioridades:
  → Emergente: 0.0%
  → Urgente: 55.0%
  → Pouco Urgente: 45.0%


***Filtra pelo quadro de manutenção predial***

In [11]:
predial = df[df['QUADRO DE TRABALHO'] == 'Predial']

## MANUTENÇÃO PREDIAL - ANÁLISE CONSOLIDADA (OTIMIZADO)

In [12]:
# ========== PREDIAL - CORRETIVAS ==========
pred_corretivas = calcular_indicadores_setor(df, 'Predial', 'Manutenção Corretiva')

corretivas_predial = pred_corretivas['df_abertas']
total_corretivas_criadas_predial = pred_corretivas['abertas']
corretivas_predial_fechadas = pred_corretivas['df_fechadas']
total_corretivas_predial_fechadas = pred_corretivas['fechadas']
percentual_corretivas_fechadas_predial = pred_corretivas['percentual']

print(f"✓ Corretivas Predial: {total_corretivas_criadas_predial} abertas, "
      f"{total_corretivas_predial_fechadas} fechadas ({percentual_corretivas_fechadas_predial}%)")


# ========== PREDIAL - PREVENTIVAS ==========
pred_preventivas = calcular_indicadores_setor(df, 'Predial', 'Manutenção Preventiva')

preventivas_predial = pred_preventivas['df_abertas']
total_preventivas_geradas_predial = pred_preventivas['abertas']
preventiva_predial_fechada = pred_preventivas['df_fechadas']
total_preventivas_predial_fechada = pred_preventivas['fechadas']
percentual_preventivas_fechadas_predial = pred_preventivas['percentual']

print(f"✓ Preventivas Predial: {total_preventivas_geradas_predial} abertas, "
      f"{total_preventivas_predial_fechada} fechadas ({percentual_preventivas_fechadas_predial}%)")


# ========== PREDIAL - PREVENTIVAS DE AR CONDICIONADO ==========
# SC GD (Santa Catarina e Gonçalves Dias)
preventivas_ar_sc_gd = preventivas_predial[preventivas_predial['PLANO'] == 'Preventiva Mensal Refrigeração SC GD']
total_preventivas_ar_sc_gd = preventivas_ar_sc_gd.shape[0]
preventivas_ar_sc_gd_fechadas = preventivas_ar_sc_gd[
    (preventivas_ar_sc_gd['ESTADO'] != 'Aberta') & 
    (preventivas_ar_sc_gd['ESTADO'] != 'Cancelada')
]
total_preventivas_ar_sc_gd_fechadas = preventivas_ar_sc_gd_fechadas.shape[0]

# CG GZ (Carangola e Goitacazes)
preventivas_ar_cg_gz = preventivas_predial[preventivas_predial['PLANO'] == 'Preventiva Mensal Refrigeração CG  GZ']
total_preventivas_ar_cg_gz = preventivas_ar_cg_gz.shape[0]
preventivas_ar_cg_gz_fechadas = preventivas_ar_cg_gz[
    (preventivas_ar_cg_gz['ESTADO'] != 'Aberta') & 
    (preventivas_ar_cg_gz['ESTADO'] != 'Cancelada')
]
total_preventivas_ar_cg_gz_fechadas = preventivas_ar_cg_gz_fechadas.shape[0]

# Demais preventivas
preventivas_demais = preventivas_predial[
    (preventivas_predial['PLANO'] != 'Preventiva Mensal Refrigeração SC GD') & 
    (preventivas_predial['PLANO'] != 'Preventiva Mensal Refrigeração CG  GZ')
]
total_preventivas_demais = preventivas_demais.shape[0]
preventivas_demais_fechadas = preventivas_demais[
    (preventivas_demais['ESTADO'] != 'Aberta') & 
    (preventivas_demais['ESTADO'] != 'Cancelada')
]
total_preventivas_demais_fechadas = preventivas_demais_fechadas.shape[0]

print(f"  → AR SC GD: {total_preventivas_ar_sc_gd_fechadas}/{total_preventivas_ar_sc_gd} fechadas")
print(f"  → AR CG GZ: {total_preventivas_ar_cg_gz_fechadas}/{total_preventivas_ar_cg_gz} fechadas")
print(f"  → Demais: {total_preventivas_demais_fechadas}/{total_preventivas_demais} fechadas")


# ========== PREDIAL - SLA POR PRIORIDADE ==========
# SLA Emergente
sla_pred_emergente = calcular_sla_por_prioridade(corretivas_predial, 'Emergente', 'Emergente')
sla_triagem_emergente_predial = sla_pred_emergente['dataframe']
sla_triagem_emergente_total_predial = sla_pred_emergente['total']
sla_triagem_emergente_regular_total_predial = sla_pred_emergente['atendidas']
percentual_sla_triagem_emergente_predial = sla_pred_emergente['perc_atendimento']
sla_fechamento_emergente_regular_total_predial = sla_pred_emergente['fechadas']
percentual_sla_fechamento_emergente_predial = sla_pred_emergente['perc_fechamento']

# Variáveis para compatibilidade
sla_triagem_emergente_regular_predial = sla_triagem_emergente_predial[
    sla_triagem_emergente_predial['Estado tempo atendimento'] == 'Regular'
]
sla_fechamento_emergente_regular_predial = sla_triagem_emergente_predial[
    sla_triagem_emergente_predial['Estado tempo fechamento'] == 'Regular'
]

print(f"\n✓ SLA Emergente: {sla_triagem_emergente_total_predial} OS → "
      f"{percentual_sla_triagem_emergente_predial}% atendimento | "
      f"{percentual_sla_fechamento_emergente_predial}% fechamento")

# SLA Urgente
sla_pred_urgente = calcular_sla_por_prioridade(corretivas_predial, 'Urgente', 'Urgente')
sla_triagem_urgente_predial = sla_pred_urgente['dataframe']
sla_triagem_urgente_predial_total = sla_pred_urgente['total']
sla_triagem_urgente_regular_total_predial = sla_pred_urgente['atendidas']
percentual_sla_triagem_urgente_predial = sla_pred_urgente['perc_atendimento']
sla_fechamento_urgente_regular_total_predial = sla_pred_urgente['fechadas']
percentual_sla_fechamento_urgente_regular_total_predial = sla_pred_urgente['perc_fechamento']

# Variáveis para compatibilidade
sla_triagem_urgente_regular_predial = sla_triagem_urgente_predial[
    sla_triagem_urgente_predial['Estado tempo atendimento'] == 'Regular'
]
sla_fechamento_urgente_regular_predial = sla_triagem_urgente_predial[
    sla_triagem_urgente_predial['Estado tempo fechamento'] == 'Regular'
]

print(f"✓ SLA Urgente: {sla_triagem_urgente_predial_total} OS → "
      f"{percentual_sla_triagem_urgente_predial}% atendimento | "
      f"{percentual_sla_fechamento_urgente_regular_total_predial}% fechamento")

# SLA Pouco Urgente
sla_pred_pouco = calcular_sla_por_prioridade(corretivas_predial, ['Pouco urgente'], 'Pouco Urgente')
sla_triagem_poucourgente_predial = sla_pred_pouco['dataframe']
sla_triagem_poucourgente_predial_total = sla_pred_pouco['total']
sla_triagem_poucourgente_regular_predial_total = sla_pred_pouco['atendidas']
percentual_sla_poucourgente_predial = sla_pred_pouco['perc_atendimento']
sla_fechamento_poucourgente_regular_total_predial = sla_pred_pouco['fechadas']
percentual_sla_fechamento_poucourgente_regular_total_predial = sla_pred_pouco['perc_fechamento']

# Variáveis para compatibilidade
sla_triagem_poucourgente_regular_predial = sla_triagem_poucourgente_predial[
    sla_triagem_poucourgente_predial['Estado tempo atendimento'] == 'Regular'
]
sla_fechamento_poucourgente_regular_predial = sla_triagem_poucourgente_predial[
    sla_triagem_poucourgente_predial['Estado tempo fechamento'] == 'Regular'
]

print(f"✓ SLA Pouco Urgente: {sla_triagem_poucourgente_predial_total} OS → "
      f"{percentual_sla_poucourgente_predial}% atendimento | "
      f"{percentual_sla_fechamento_poucourgente_regular_total_predial}% fechamento")


# ========== PREDIAL - DISTRIBUIÇÃO DE PRIORIDADES ==========
# Recalcular Percentagem_emergente e percentagem_urgente para Predial
Percentagem_emergente = round((sla_triagem_emergente_total_predial / total_corretivas_criadas_predial) * 100, 2) if total_corretivas_criadas_predial > 0 else 0
percentagem_urgente = round((sla_triagem_urgente_predial_total / total_corretivas_criadas_predial) * 100, 2) if total_corretivas_criadas_predial > 0 else 0
percentagem_poucourgente = round((sla_triagem_poucourgente_predial_total / total_corretivas_criadas_predial) * 100, 2) if total_corretivas_criadas_predial > 0 else 0

print(f"\n✓ Distribuição de Prioridades Predial:")
print(f"  → Emergente: {Percentagem_emergente}%")
print(f"  → Urgente: {percentagem_urgente}%")
print(f"  → Pouco Urgente: {percentagem_poucourgente}%")

✓ Corretivas Predial: 73 abertas, 70 fechadas (95.89%)
✓ Preventivas Predial: 353 abertas, 191 fechadas (54.11%)
  → AR SC GD: 40/157 fechadas
  → AR CG GZ: 85/85 fechadas
  → Demais: 66/111 fechadas

✓ SLA Emergente: 12 OS → 33.33% atendimento | 16.67% fechamento
✓ SLA Urgente: 26 OS → 100.0% atendimento | 57.69% fechamento
✓ SLA Pouco Urgente: 35 OS → 94.29% atendimento | 68.57% fechamento

✓ Distribuição de Prioridades Predial:
  → Emergente: 16.44%
  → Urgente: 35.62%
  → Pouco Urgente: 47.95%


***AR CONDICIONADOS PENDENTES DE PREVENTIVA NO MÊS***

In [13]:
# Configurando a exibição do pandas para mostrar todas as colunas
pd.set_option('display.max_columns', None)  # Exibe todas as colunas
pd.set_option('display.expand_frame_repr', False)  # Evita que as colunas quebrem em múltiplas linhas
pd.set_option('display.max_rows', 100)  # Mostra até 100 linhas (ajuste conforme necessário)

# Combinar os dois DataFrames de ar condicionado
preventivas_arcondicionado = pd.concat([preventivas_ar_sc_gd, preventivas_ar_cg_gz], ignore_index=True)

# Filtrando os dados
preventivas_arcondicionado_abertas = preventivas_arcondicionado[preventivas_arcondicionado['ESTADO'] == 'Aberta']

# Mostrando o DataFrame filtrado de maneira completa
print(preventivas_arcondicionado_abertas)

        ID  NUMERO           TIPO SERVIÇO  ESTADO                            SOLICITANTE LOCALIZAÇÃO       RESPONSAVEL      DATA DE CRIAÇÃO DATA DE CONCLUSÃO   PRIORIDADE  PRAZO                                 PLANO OBSERVAÇÕES DESCRIÇÃO DO SERVIÇO QUADRO DE TRABALHO Horário do início do atendimento Tempo até o início de atendimento (horas:minutos:segundos) Estado tempo atendimento Tempo total do serviço (horas:minutos:segundos) Estado tempo fechamento  Tempo de atendimento/ Tempo total do serviço (%)
4    21980   21098  Manutenção Preventiva  Aberta  Consultoria - Marcação Cirúrgica – SC         NaN  Gabriel da Silva  01/12/2025 03:20:29               NaN  Não urgente    NaN  Preventiva Mensal Refrigeração SC GD         NaN                  NaN            Predial                              NaN                                                NaN                         Atrasado                                             NaN                Atrasado                                     

---
## ANÁLISE DE DESEMPENHO POR TÉCNICO
Extração de dados de desempenho individual dos técnicos, incluindo tipo de serviço e quantidade de OS realizadas.

In [14]:
# Nome da coluna de responsável (ajuste conforme necessário)
coluna_responsavel = 'RESPONSAVEL'  # Pode ser 'TÉCNICO', 'RESPONSAVEL', etc.

# Filtrar OS realizadas (excluindo Aberta e Cancelada)
df_fechadas = df[(df['ESTADO'] != 'Aberta') & (df['ESTADO'] != 'Cancelada')].copy()

# Criar tabela de desempenho por técnico
desempenho_tecnico = df_fechadas.groupby([coluna_responsavel, 'TIPO SERVIÇO']).size().unstack(fill_value=0)
desempenho_tecnico['Total OS'] = desempenho_tecnico.sum(axis=1)
desempenho_tecnico = desempenho_tecnico.sort_values('Total OS', ascending=False)

print("=" * 80)
print("DESEMPENHO POR TÉCNICO - ORDENS DE SERVIÇO FECHADAS")
print("=" * 80)
print(desempenho_tecnico)
print(f"\nTotal de técnicos: {len(desempenho_tecnico)}")

DESEMPENHO POR TÉCNICO - ORDENS DE SERVIÇO FECHADAS
TIPO SERVIÇO               Analise de Infraestrutura  Busca Ativa  Manutenção Corretiva  Manutenção Preventiva  Teste de Segurança Elétrica  Vistoria Diária   Total OS
RESPONSAVEL                                                                                                                                                            
Diego Mendes                                       0            0                    15                     68                           37                 0       120
Daniel Pereira                                     0            0                     5                     91                            0                 0        96
Lara Soares                                        0            8                    12                      4                            7                38        69
Guilherme Breno Magalhaes                          1            9                    19                     

***Desempenho detalhado por técnico - Engenharia Clínica***

In [15]:
# Desempenho dos técnicos de Engenharia Clínica
df_engenharia_fechadas = df_fechadas[df_fechadas['QUADRO DE TRABALHO'] == 'Engenharia Clínica']

# Criar resumo por tipo de serviço
desempenho_engenharia = df_engenharia_fechadas.groupby([coluna_responsavel, 'TIPO SERVIÇO']).size().unstack(fill_value=0)
desempenho_engenharia['Total OS'] = desempenho_engenharia.sum(axis=1)
desempenho_engenharia = desempenho_engenharia.sort_values('Total OS', ascending=False)

print("=" * 80)
print("DESEMPENHO - TÉCNICOS DE ENGENHARIA CLÍNICA")
print("=" * 80)
print(desempenho_engenharia)
print(f"\nTotal de técnicos de Engenharia: {len(desempenho_engenharia)}")

DESEMPENHO - TÉCNICOS DE ENGENHARIA CLÍNICA
TIPO SERVIÇO  Busca Ativa  Manutenção Corretiva  Manutenção Preventiva  Teste de Segurança Elétrica  Vistoria Diária   Total OS
RESPONSAVEL                                                                                                                    
Diego Mendes            0                     9                     61                           37                 0       107
Lara Soares             8                    10                      3                            6                36        63

Total de técnicos de Engenharia: 2


***Desempenho detalhado por técnico - Manutenção Predial***

In [16]:
# Desempenho dos técnicos de Manutenção Predial
df_predial_fechadas = df_fechadas[df_fechadas['QUADRO DE TRABALHO'] == 'Predial']

# Criar resumo por tipo de serviço
desempenho_predial = df_predial_fechadas.groupby([coluna_responsavel, 'TIPO SERVIÇO']).size().unstack(fill_value=0)
desempenho_predial['Total OS'] = desempenho_predial.sum(axis=1)
desempenho_predial = desempenho_predial.sort_values('Total OS', ascending=False)

print("=" * 80)
print("DESEMPENHO - TÉCNICOS DE MANUTENÇÃO PREDIAL")
print("=" * 80)
print(desempenho_predial)
print(f"\nTotal de técnicos de Predial: {len(desempenho_predial)}")

DESEMPENHO - TÉCNICOS DE MANUTENÇÃO PREDIAL
TIPO SERVIÇO               Analise de Infraestrutura  Busca Ativa  Manutenção Corretiva  Manutenção Preventiva  Vistoria Diária   Total OS
RESPONSAVEL                                                                                                                               
Daniel Pereira                                     0            0                     5                     90                 0        95
Guilherme Breno Magalhaes                          1            9                    19                     34                 0        63
Gabriel da Silva                                   1            0                     4                     40                 0        45
Leonardo Oliveira                                  1            3                    34                      0                 0        38
Diego Mendes                                       0            0                     6                      7            

***Análise de SLA por técnico - Atendimento dentro do prazo***

In [17]:
# Análise de SLA por técnico (apenas corretivas fechadas)
df_corretivas_fechadas = df_fechadas[df_fechadas['TIPO SERVIÇO'] == 'Manutenção Corretiva']

# Calcular total de OS e OS dentro do prazo por técnico
sla_por_tecnico = df_corretivas_fechadas.groupby(coluna_responsavel).agg({
    'ESTADO': 'count',  # Total de OS fechadas
    'Estado tempo atendimento': lambda x: (x == 'Regular').sum(),  # Atendidas no prazo
    'Estado tempo fechamento': lambda x: (x == 'Regular').sum()  # Fechadas no prazo
}).rename(columns={
    'ESTADO': 'Total Corretivas Fechadas',
    'Estado tempo atendimento': 'Atendidas no Prazo',
    'Estado tempo fechamento': 'Fechadas no Prazo'
})

# Calcular percentuais
sla_por_tecnico['% Atendimento no Prazo'] = round((sla_por_tecnico['Atendidas no Prazo'] / sla_por_tecnico['Total Corretivas Fechadas']) * 100, 2)
sla_por_tecnico['% Fechamento no Prazo'] = round((sla_por_tecnico['Fechadas no Prazo'] / sla_por_tecnico['Total Corretivas Fechadas']) * 100, 2)

sla_por_tecnico = sla_por_tecnico.sort_values('Total Corretivas Fechadas', ascending=False)

print("=" * 100)
print("ANÁLISE DE SLA POR TÉCNICO - CORRETIVAS")
print("=" * 100)
print(sla_por_tecnico)

ANÁLISE DE SLA POR TÉCNICO - CORRETIVAS
                           Total Corretivas Fechadas  Atendidas no Prazo  Fechadas no Prazo  % Atendimento no Prazo  % Fechamento no Prazo
RESPONSAVEL                                                                                                                               
Leonardo Oliveira                                 34                  30                 17                   88.24                  50.00
Guilherme Breno Magalhaes                         19                  19                 15                  100.00                  78.95
Diego Mendes                                      15                  12                 10                   80.00                  66.67
Lara Soares                                       12                   8                  6                   66.67                  50.00
Daniel Pereira                                     5                   1                  0                   20.00           

***Ranking dos técnicos - Top 10 por quantidade de OS***

In [18]:
# Top 10 técnicos por quantidade de OS fechadas
top_10_tecnicos = df_fechadas.groupby(coluna_responsavel).size().sort_values(ascending=False).head(10)

print("=" * 60)
print("TOP 10 TÉCNICOS - QUANTIDADE DE OS FECHADAS")
print("=" * 60)
for i, (tecnico, qtd) in enumerate(top_10_tecnicos.items(), 1):
    print(f"{i:2}. {tecnico}: {qtd} OS")

TOP 10 TÉCNICOS - QUANTIDADE DE OS FECHADAS
 1. Diego Mendes: 120 OS
 2. Daniel Pereira: 96 OS
 3. Lara Soares: 69 OS
 4. Guilherme Breno Magalhaes: 63 OS
 5. Gabriel da Silva: 45 OS
 6. Leonardo Oliveira: 38 OS
 7. Rafael Ribeiro: 2 OS


***Preparar dados de técnicos para exportação***

In [19]:
# Forçar Diego e Lara para Engenharia Clínica
df_fechadas_ajustado = df_fechadas.copy()
df_fechadas_ajustado.loc[df_fechadas_ajustado[coluna_responsavel].isin(['Diego Mendes', 'Lara Soares']), 'QUADRO DE TRABALHO'] = 'Engenharia Clínica'

# Criar DataFrame detalhado de desempenho por técnico para consolidação
df_desempenho_tecnicos = df_fechadas_ajustado.groupby([coluna_responsavel, 'QUADRO DE TRABALHO']).agg({
    'TIPO SERVIÇO': [
        ('Corretivas', lambda x: (x == 'Manutenção Corretiva').sum()),
        ('Preventivas', lambda x: (x == 'Manutenção Preventiva').sum())
    ],
    'ESTADO': 'count',
    'Estado tempo atendimento': lambda x: (x == 'Regular').sum(),
    'Estado tempo fechamento': lambda x: (x == 'Regular').sum()
})

# Achatar colunas multi-nível
df_desempenho_tecnicos.columns = ['Corretivas', 'Preventivas', 'Total OS', 'Atendidas no Prazo', 'Fechadas no Prazo']
df_desempenho_tecnicos = df_desempenho_tecnicos.reset_index()

# Adicionar percentuais
df_desempenho_tecnicos['% Atendimento Prazo'] = round((df_desempenho_tecnicos['Atendidas no Prazo'] / df_desempenho_tecnicos['Total OS']) * 100, 2)
df_desempenho_tecnicos['% Fechamento Prazo'] = round((df_desempenho_tecnicos['Fechadas no Prazo'] / df_desempenho_tecnicos['Total OS']) * 100, 2)

# Criar dicionário de indicadores por técnico para adicionar ao CSV consolidado
indicadores_tecnicos = {}

for _, row in df_desempenho_tecnicos.iterrows():
    tecnico = row[coluna_responsavel]
    setor = row['QUADRO DE TRABALHO']
    prefixo = f"TEC - {tecnico}"
    
    indicadores_tecnicos[f"{prefixo} - Setor"] = setor
    indicadores_tecnicos[f"{prefixo} - Corretivas"] = row['Corretivas']
    indicadores_tecnicos[f"{prefixo} - Preventivas"] = row['Preventivas']
    indicadores_tecnicos[f"{prefixo} - Total OS"] = row['Total OS']
    indicadores_tecnicos[f"{prefixo} - Atendidas no Prazo"] = row['Atendidas no Prazo']
    indicadores_tecnicos[f"{prefixo} - Fechadas no Prazo"] = row['Fechadas no Prazo']
    indicadores_tecnicos[f"{prefixo} - % Atend. Prazo"] = row['% Atendimento Prazo']
    indicadores_tecnicos[f"{prefixo} - % Fech. Prazo"] = row['% Fechamento Prazo']

print(f"DataFrame de desempenho dos técnicos preparado:")
print(df_desempenho_tecnicos.to_string())
print(f"\nTotal de indicadores de técnicos gerados: {len(indicadores_tecnicos)}")

DataFrame de desempenho dos técnicos preparado:
                 RESPONSAVEL  QUADRO DE TRABALHO  Corretivas  Preventivas  Total OS  Atendidas no Prazo  Fechadas no Prazo  % Atendimento Prazo  % Fechamento Prazo
0             Daniel Pereira             Predial           5           90        95                   1                  0                 1.05                0.00
1               Diego Mendes  Engenharia Clínica          15           68       120                  12                 10                10.00                8.33
2           Gabriel da Silva             Predial           4           40        45                   3                  3                 6.67                6.67
3  Guilherme Breno Magalhaes             Predial          19           34        63                  19                 24                30.16               38.10
4                Lara Soares  Engenharia Clínica          12            4        69                   8                  6          

***CONSOLIDAÇÃO DE INDICADORES - EXPORTAÇÃO PARA CSV***

**Como usar:**
1. Para processar outro mês, altere apenas a célula 2 com o novo nome do arquivo
2. Execute todo o notebook (Run All)
3. O CSV será atualizado automaticamente com o novo mês em uma nova linha
4. Os meses serão ordenados cronologicamente no arquivo final

In [20]:
import re
import os

# Extrair o mês do nome do arquivo (funciona com "servico" ou "serviço")
nome_arquivo = os.path.basename(caminho_arquivo)
match = re.search(r'ordens_servi[cç]o_(\w+)\.xls', nome_arquivo)
mes = match.group(1).capitalize() if match else "Desconhecido"

print(f"Mês identificado: {mes}")

Mês identificado: Dezembro


In [21]:
# Criar dicionário com todos os indicadores calculados
indicadores = {
    'Mês': mes,
    
    # Indicadores Gerais
    'Total Corretivas Abertas': total_corretivas_abertas,
    'Total Corretivas Fechadas': total_corretivas_fechadas,
    'Total Preventivas Abertas': total_preventivas_abertas,
    'Total Preventivas Fechadas': total_preventivas_fechadas,
    
    # Engenharia Clínica - Corretivas
    'ENG - Total Corretivas Abertas': total_corretivas_criadas_engenharia,
    'ENG - Total Corretivas Fechadas': total_corretivas_engenharia_fechadas,
    'ENG - % Corretivas Fechadas': round(percentual_corretivas_fechadas_engenharia, 2),
    'ENG - Corretivas Atendidas no Prazo': total_corretivas_engenharia_fechadas_regular,
    'ENG - % Corretivas Atendidas no Prazo': round(percentual_corretivas_fechadas_engenharia_regular, 2),
    
    # Engenharia Clínica - Preventivas
    'ENG - Total Preventivas Abertas': total_preventivas_geradas_engenharia,
    'ENG - Total Preventivas Fechadas': total_preventivas_engenharia_fechadas,
    'ENG - % Preventivas Fechadas': round(percentual_preventivas_fechadas_engenharia, 2),
    
    # Engenharia Clínica - SLA Emergente
    'ENG - OS Emergentes': sla_triagem_emergente_total_engenharia,
    'ENG - Emergente Atendidas no Prazo': sla_triagem_emergente_regular_total_engenharia,
    'ENG - % SLA Triagem Emergente': round(percentual_sla_triagem_emergente_engenharia, 2) if sla_triagem_emergente_regular_total_engenharia > 0 else 0,
    'ENG - Emergente Fechadas no Prazo': sla_fechamento_emergente_regular_total_engenharia,
    'ENG - % SLA Fechamento Emergente': round(percentual_sla_fechamento_emergente_engenharia, 2) if sla_fechamento_emergente_regular_total_engenharia > 0 else 0,
    
    # Engenharia Clínica - SLA Urgente
    'ENG - OS Urgentes': sla_triagem_urgente_total_engenharia,
    'ENG - Urgente Atendidas no Prazo': sla_triagem_urgente_regular_total_engenharia,
    'ENG - % SLA Triagem Urgente': round(percentual_sla_triagem_urgente_engenharia, 2) if sla_triagem_urgente_total_engenharia > 0 else 0,
    'ENG - Urgente Fechadas no Prazo': sla_fechamento_urgente_regular_total_engenharia,
    'ENG - % SLA Fechamento Urgente': round(percentual_sla_fechamento_urgente_regular_total_engenharia, 2) if sla_triagem_urgente_total_engenharia > 0 else 0,
    
    # Engenharia Clínica - SLA Pouco Urgente
    'ENG - OS Pouco Urgentes': sla_triagem_poucourgente_engenharia_total,
    'ENG - Pouco Urgente Atendidas no Prazo': sla_triagem_poucourgente_regular_engenharia_total,
    'ENG - % SLA Triagem Pouco Urgente': round(percentual_sla_poucourgente_engenharia, 2) if sla_triagem_poucourgente_engenharia_total > 0 else 0,
    'ENG - Pouco Urgente Fechadas no Prazo': sla_fechamento_poucourgente_regular_total_engenharia,
    'ENG - % SLA Fechamento Pouco Urgente': round(percentual_sla_fechamento_poucourgente_regular_total_engenharia, 2) if sla_triagem_poucourgente_engenharia_total > 0 else 0,
    
    # Engenharia Clínica - Distribuição Prioridades
    'ENG - Qtd Emergentes': sla_triagem_emergente_total_engenharia,
    'ENG - Qtd Urgentes': sla_triagem_urgente_total_engenharia,
    'ENG - Qtd Pouco Urgentes': sla_triagem_poucourgente_engenharia_total,
    'ENG - % Emergentes': round(Percentagem_emergente, 2),
    'ENG - % Urgentes': round(percentagem_urgente, 2),
    'ENG - % Pouco Urgentes': round(percentagem_poucourgente, 2),
    
    # Predial - Corretivas
    'PRED - Total Corretivas Abertas': total_corretivas_criadas_predial,
    'PRED - Total Corretivas Fechadas': total_corretivas_predial_fechadas,
    'PRED - % Corretivas Fechadas': round(percentual_corretivas_fechadas_predial, 2),
    
    # Predial - Preventivas
    'PRED - Total Preventivas Abertas': total_preventivas_geradas_predial,
    'PRED - Total Preventivas Fechadas': total_preventivas_predial_fechada,
    'PRED - % Preventivas Fechadas': round(percentual_preventivas_fechadas_predial, 2),
    'PRED - Preventivas AR SC GD Abertas': total_preventivas_ar_sc_gd,
    'PRED - Preventivas AR SC GD Fechadas': total_preventivas_ar_sc_gd_fechadas,
    'PRED - Preventivas AR CG GZ Abertas': total_preventivas_ar_cg_gz,
    'PRED - Preventivas AR CG GZ Fechadas': total_preventivas_ar_cg_gz_fechadas,
    'PRED - Demais Preventivas Abertas': total_preventivas_demais,
    'PRED - Demais Preventivas Fechadas': total_preventivas_demais_fechadas,
    
    # Predial - SLA Emergente
    'PRED - OS Emergentes': sla_triagem_emergente_total_predial,
    'PRED - Emergente Atendidas no Prazo': sla_triagem_emergente_regular_total_predial,
    'PRED - % SLA Triagem Emergente': round(percentual_sla_triagem_emergente_predial, 2) if sla_triagem_emergente_regular_total_predial > 0 else 0,
    'PRED - Emergente Fechadas no Prazo': sla_fechamento_emergente_regular_total_predial,
    'PRED - % SLA Fechamento Emergente': round(percentual_sla_fechamento_emergente_predial, 2) if sla_fechamento_emergente_regular_total_predial > 0 else 0,
    
    # Predial - SLA Urgente
    'PRED - OS Urgentes': sla_triagem_urgente_predial_total,
    'PRED - Urgente Atendidas no Prazo': sla_triagem_urgente_regular_total_predial,
    'PRED - % SLA Triagem Urgente': round(percentual_sla_triagem_urgente_predial, 2),
    'PRED - Urgente Fechadas no Prazo': sla_fechamento_urgente_regular_total_predial,
    'PRED - % SLA Fechamento Urgente': round(percentual_sla_fechamento_urgente_regular_total_predial, 2),
    
    # Predial - SLA Pouco Urgente
    'PRED - OS Pouco Urgentes': sla_triagem_poucourgente_predial_total,
    'PRED - Pouco Urgente Atendidas no Prazo': sla_triagem_poucourgente_regular_predial_total,
    'PRED - % SLA Triagem Pouco Urgente': round(percentual_sla_poucourgente_predial, 2),
    'PRED - Pouco Urgente Fechadas no Prazo': sla_fechamento_poucourgente_regular_total_predial,
    'PRED - % SLA Fechamento Pouco Urgente': round(percentual_sla_fechamento_poucourgente_regular_total_predial, 2),
}

# Adicionar indicadores de desempenho dos técnicos ao dicionário principal
indicadores.update(indicadores_tecnicos)

print(f"Total de indicadores coletados: {len(indicadores) - 1}")
print(f"  - Indicadores gerais/setoriais: {len(indicadores) - len(indicadores_tecnicos) - 1}")
print(f"  - Indicadores de técnicos: {len(indicadores_tecnicos)}")

Total de indicadores coletados: 116
  - Indicadores gerais/setoriais: 60
  - Indicadores de técnicos: 56


In [22]:
# Criar DataFrame com uma linha (representando o mês atual)
df_indicadores = pd.DataFrame([indicadores])

# Exibir preview
print("Preview dos indicadores:")
print(df_indicadores.head())

Preview dos indicadores:
        Mês  Total Corretivas Abertas  Total Corretivas Fechadas  Total Preventivas Abertas  Total Preventivas Fechadas  ENG - Total Corretivas Abertas  ENG - Total Corretivas Fechadas  ENG - % Corretivas Fechadas  ENG - Corretivas Atendidas no Prazo  ENG - % Corretivas Atendidas no Prazo  ENG - Total Preventivas Abertas  ENG - Total Preventivas Fechadas  ENG - % Preventivas Fechadas  ENG - OS Emergentes  ENG - Emergente Atendidas no Prazo  ENG - % SLA Triagem Emergente  ENG - Emergente Fechadas no Prazo  ENG - % SLA Fechamento Emergente  ENG - OS Urgentes  ENG - Urgente Atendidas no Prazo  ENG - % SLA Triagem Urgente  ENG - Urgente Fechadas no Prazo  ENG - % SLA Fechamento Urgente  ENG - OS Pouco Urgentes  ENG - Pouco Urgente Atendidas no Prazo  ENG - % SLA Triagem Pouco Urgente  ENG - Pouco Urgente Fechadas no Prazo  ENG - % SLA Fechamento Pouco Urgente  ENG - Qtd Emergentes  ENG - Qtd Urgentes  ENG - Qtd Pouco Urgentes  ENG - % Emergentes  ENG - % Urgentes  

In [23]:
# Caminho do arquivo CSV consolidado (relativo ao projeto atual)
arquivo_csv_consolidado = 'c:/Users/rafae/Documents/DSH/Clientes/COMG/Indicadores/Codigo/frontend/public/indicadores_consolidados.csv'

# Criar diretório se não existir
diretorio_destino = os.path.dirname(arquivo_csv_consolidado)
if not os.path.exists(diretorio_destino):
    os.makedirs(diretorio_destino)
    print(f"Diretório criado: {diretorio_destino}")

# Verificar se o arquivo já existe
if os.path.exists(arquivo_csv_consolidado):
    # Ler arquivo existente
    df_existente = pd.read_csv(arquivo_csv_consolidado)
    
    # Verificar se o mês já existe
    if mes in df_existente['Mês'].values:
        # Atualizar a linha do mês existente
        df_existente = df_existente[df_existente['Mês'] != mes]
        df_final = pd.concat([df_existente, df_indicadores], ignore_index=True)
        print(f"Mês '{mes}' atualizado no arquivo existente.")
    else:
        # Adicionar nova linha
        df_final = pd.concat([df_existente, df_indicadores], ignore_index=True)
        print(f"Mês '{mes}' adicionado ao arquivo existente.")
else:
    # Criar novo arquivo
    df_final = df_indicadores
    print(f"Novo arquivo criado com o mês '{mes}'.")

# Ordenar por mês (opcional - você pode criar uma ordem customizada)
meses_ordem = ['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 
               'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']
df_final['Ordem'] = df_final['Mês'].apply(lambda x: meses_ordem.index(x) if x in meses_ordem else 999)
df_final = df_final.sort_values('Ordem').drop('Ordem', axis=1)

# Salvar no CSV
df_final.to_csv(arquivo_csv_consolidado, index=False, encoding='utf-8-sig')
print(f"\nArquivo salvo com sucesso em: {arquivo_csv_consolidado}")
print(f"Total de meses no arquivo: {len(df_final)}")

Mês 'Dezembro' adicionado ao arquivo existente.

Arquivo salvo com sucesso em: c:/Users/rafae/Documents/DSH/Clientes/COMG/Indicadores/Codigo/frontend/public/indicadores_consolidados.csv
Total de meses no arquivo: 12


***Visualizar resumo do arquivo consolidado***

In [24]:
# Exibir as primeiras colunas do arquivo consolidado
print("Resumo do arquivo consolidado:")
print(f"Dimensões: {df_final.shape[0]} meses x {df_final.shape[1]} indicadores")
print("\nPrimeiras colunas:")
print(df_final.iloc[:, :10].to_string())

Resumo do arquivo consolidado:
Dimensões: 12 meses x 135 indicadores

Primeiras colunas:
          Mês  Total Corretivas Abertas  Total Corretivas Fechadas  Total Preventivas Abertas  Total Preventivas Fechadas  ENG - Total Corretivas Abertas  ENG - Total Corretivas Fechadas  ENG - % Corretivas Fechadas  ENG - Corretivas Atendidas no Prazo  ENG - % Corretivas Atendidas no Prazo
0     Janeiro                       220                        217                        452                         299                              72                               72                       100.00                                   53                                  73.61
1   Fevereiro                       160                        158                        455                         353                              56                               55                        98.21                                   29                                  51.79
2       Março                      